In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import bs4
import requests
import json

In [3]:
# Downloading and parsing the data into a Python dict

team_dict = {}

url = 'http://www.espnfc.com/barclays-premier-league/23/table'
r  = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser') 
# Note: html5lib deals better with broken html than lxml

for td in soup.findAll('td', { 'class' : 'pos' }):
    rank = int(td.text)
    res = [i.text for i in td.next_siblings if isinstance(i, bs4.element.Tag) and i.text!='\xa0']
    team_name = res[0].strip()
    print team_name, rank
    
    values = []
    for i in res[1:]:
        a = i.replace(u'\xa0', '0')
        values.append(int(a))
    
    team_dict[team_name] = [rank] + values


for i in team_dict:
    remove_val = [1, 8, 14, 20]
    team = team_dict[i]
    shift = 0 
    for j in remove_val:
       
        team.pop(j + shift)
        shift = shift - 1
    
    


df = pd.DataFrame.from_dict(team_dict, orient='index')
cols = ['Pos','P_ov','W_ov','D_ov','L_ov','F_ov','A_ov',
            'W_hm','D_hm','L_hm','F_hm','A_hm', 'W_aw',
            'D_aw','L_aw','F_aw','A_aw','GD','PTS']
df.columns = cols
df = df.sort('Pos')
df['team'] = df.index
df = df[['team']+cols]
df


df.to_csv('./fantasyfootballscout.csv', index=False)

Chelsea 1
Tottenham Hotspur 2
Manchester City 3
Liverpool 4
Manchester United 5
Arsenal 6
Everton 7
West Bromwich Albion 8
Stoke City 9
Southampton 10
AFC Bournemouth 11
West Ham United 12
Burnley 13
Watford 14
Leicester City 15
Crystal Palace 16
Swansea City 17
Hull City 18
Middlesbrough 19
Sunderland 20


/Users/gorkhali/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: UnicodeWarning: Unicode unequal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/gorkhali/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:41: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [4]:
import matplotlib 
matplotlib.use('agg')

import matplotlib.pyplot as plt
import pylab as pl
import numpy as np

#import numpy as np
#import matplotlib.pyplot as plt
#plt.ioff()

url = "https://fantasy.premierleague.com/drf/bootstrap-static"
r  = requests.get(url)
data = json.loads(r.text)
all_players = data['elements']

player_dict = {}
for i in all_players:
    player_dict[i['id']] = i['web_name']
    


del(player_dict[652])
del(player_dict[653])
del(player_dict[654])

print player_dict


/Users/gorkhali/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


{1: u'Ospina', 2: u'Cech', 3: u'Koscielny', 4: u'Mertesacker', 5: u'Gabriel', 6: u'Beller\xedn', 7: u'Gibbs', 8: u'Debuchy', 9: u'Jenkinson', 10: u'Monreal', 11: u'Chambers', 12: u'S\xe1nchez', 13: u'Walcott', 14: u'\xd6zil', 15: u'Cazorla', 16: u'Ramsey', 17: u'Wilshere', 18: u'Chamberlain', 19: u'Gnabry', 20: u'Coquelin', 21: u'Iwobi', 22: u'Elneny', 23: u'Campbell', 24: u'Xhaka', 25: u'Giroud', 26: u'Welbeck', 27: u'Sanogo', 28: u'Asano', 29: u'Boruc', 30: u'Federici', 31: u'Francis', 32: u'Cook', 33: u'Daniels', 34: u'Smith', 35: u'Mings', 36: u'Ak\xe9', 37: u'Arter', 38: u'Pugh', 39: u'Surman', 40: u"O'Kane", 41: u'MacDonald', 42: u'Stanislas', 43: u'Gosling', 44: u'Gradel', 45: u'Hyndman', 46: u'Ibe', 47: u'Cook', 48: u'King', 49: u'Callum Wilson', 50: u'Rantie', 51: u'Grabban', 52: u'Afobe', 53: u'Mousset', 54: u'Heaton', 55: u'Lowton', 56: u'Mee', 57: u'Ward', 58: u'Keane', 59: u'Tarkowski', 60: u'Darikwa', 61: u'Jones', 62: u'Boyd', 63: u'Arfield', 64: u'Marney', 65: u'Ulvesta

In [16]:
import sys
from time import time
from sklearn.naive_bayes import GaussianNB


scorers = []
possible_scorers = []
count = 0
for i in player_dict:
    print i, player_dict[i]

    url = "https://fantasy.premierleague.com/drf/element-summary/" + str(i)
    r  = requests.get(url)
    data = json.loads(r.text)
    # data['fixtures']
    value = data['history']

    # print value
    all_features = []
    labels = []
    a = False;
    for j in value:
#         print i['total_points'],i['kickoff_time_formatted']
        features = [int(j['goals_scored']), int(j['total_points']),int(j['clean_sheets'])]
        all_features.append(features)
        if a:
            if j['total_points']>=8:
                labels.append(1)
            else:
                labels.append(0)
        a = True


    first_test = all_features.pop()
    first_test = all_features.pop()
    label_test = labels.pop()
    print len(all_features)
    print len(labels)
    print first_test

    features_train = all_features
    labels_train = labels
    
    #########################################################
    ### your code goes here ###

    clf = GaussianNB()
    t0 = time()
    clf.fit(features_train, labels_train)
    print "training time:", round(time()-t0, 3), "s"

    
    res = clf.predict(first_test)
    
    if res[0] == 1:
        possible_scorers.append(player_dict[i])
        
        
    if res[0] == label_test:
        print True
        count += 1
        if res[0] == 1:
            print i
            scorers.append(player_dict[i])
        
print scorers
print possible_scorers

1 Ospina
25
25
[0, 0, 0]
training time: 0.003 s
True
2 Cech


/Users/gorkhali/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


25
25
[0, 2, 0]
training time: 0.002 s
True
3 Koscielny


/Users/gorkhali/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


25
25
[0, 1, 0]
training time: 0.001 s
4 Mertesacker


...

/Users/gorkhali/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/gorkhali/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


5
5
[0, 0, 0]
training time: 0.001 s
True
649 Cargill
4
4
[0, 0, 0]
training time: 0.0 s
True
650 Hassen


/Users/gorkhali/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/gorkhali/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


1
1
[0, 0, 0]
training time: 0.001 s
True
651 Agyei
2
2
[0, 1, 0]
training time: 0.0 s
True
[u'King', u'Heaton', u'Willian', u'Mahrez', u'Sakho', u'Ward-Prowse', u'Eriksen']
[u'Koscielny', u'Chambers', u'Chamberlain', u'Iwobi', u'Boruc', u'Cook', u'Daniels', u'Ak\xe9', u'Arter', u'Stanislas', u'Gosling', u'King', u'Heaton', u'Keane', u'F\xe0bregas', u'Willian', u'Moses', u'Dann', u'Ward', u'McArthur', u'Baines', u'Coleman', u'Dawson', u'Snodgrass', u'Maloney', u'Schmeichel', u'Mahrez', u'Sakho', u'Lovren', u'Clyne', u'Henderson', u'Man\xe9', u'Kolarov', u'Zabaleta', u'Clichy', u'Silva', u'G\xfcndogan', u'Iheanacho', u'de Gea', u'Jones', u'Shaw', u'Blind', u'Martial', u'Vald\xe9s', u'Gibson', u'Bertrand', u'van Dijk', u'Redmond', u'Ward-Prowse', u'Given', u'Shawcross', u'Adam', u'Whelan', u'Arnautovic', u'Bojan', u'Kaboul', u'Jones', u'Kon\xe9', u'Fern\xe1ndez', u'Sigurdsson', u'Fer', u'Lloris', u'Rose', u'Trippier', u'Chadli', u'Eriksen', u'Alli', u'Wanyama', u'Son', u'Janssen', u'Nyom

/Users/gorkhali/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/gorkhali/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [10]:

print count/651.0 * 100


86.0215053763
